In [ ]:
'''
Introdução à Econometria - Uma abordagem moderna (Tradução da 6 edição norte-americana)
Autor: WOOLDRIDGE, J. M.
Editora: CENGAGE LEARNING

Cap. 8: Heterocedasticidade (Heteroskedasticity)
Exemplo 8.9: Determinantes da propriedade de computadores pessoais
             (DETERMINANTS OF PERSONAL COMPUTER OWNERSHIP)
             
Arquivo com os dados: gpa1.xls

Arquivo com dados em:
http://students.cengage.com.br/dashboard/private/livroView.jsf;jsessionid=95E9AD889A4A4B7ABBD2A5251F1E14BE?id=104577

Em caso de dúvidas ou problemas, solicitamos, por gentileza, entrar em contato pelo e-mail:
python.economia@gmail.com
'''

In [1]:
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
df = pd.read_excel('gpa1.xls',
                   header=None,
                   usecols=[10, 11, 18, 27, 28],
                   names=['hsGPA', 'ACT', 'PC', 'fathcoll', 'mothcoll'])

In [3]:
df.head()

,hsGPA,ACT,PC,fathcoll,mothcoll
0,3.0,21,0,0,0
1,3.2,24,0,1,1
2,3.6,26,0,1,1
3,3.5,27,0,0,0
4,3.9,28,0,1,0


In [4]:
# gera a coluna 'parcoll': indicador binário igual a um se pelo menos um dos genitores tem curso superior
df['parcoll'] = df.loc[:, ['fathcoll', 'mothcoll']].apply(max, axis=1)

In [5]:
df.head()

,hsGPA,ACT,PC,fathcoll,mothcoll,parcoll
0,3.0,21,0,0,0,0
1,3.2,24,0,1,1,1
2,3.6,26,0,1,1,1
3,3.5,27,0,0,0,0
4,3.9,28,0,1,0,1


### Solução sugerida

In [6]:
# Regressão pelo Método de Mínimos Quadrados Ordinários (MQO ou OLS na sigla em inglês)
modelo_ols = smf.ols('PC ~ hsGPA + ACT + parcoll', data=df)
reg_ols = modelo_ols.fit()

In [7]:
tabela = pd.DataFrame({'estimativas MQO': round(reg_ols.params, 4), 
                       'nonrobust SE': round(reg_ols.bse, 3), 
                       'robust SE': round(reg_ols.HC0_se, 3)})

print('Exemplo 8.9 [Variável dependente: PC]\n')
print(tabela)
print(f'\nObservações: {int(reg_ols.nobs)}')
print(f'R-quadrado: {round(reg_ols.rsquared, 4)}')

Exemplo 8.9 [Variável dependente: PC]

           estimativas MQO  nonrobust SE  robust SE
Intercept          -0.0004         0.491      0.489
hsGPA               0.0654         0.137      0.139
ACT                 0.0006         0.015      0.016
parcoll             0.2211         0.093      0.087

Observações: 141
R-quadrado: 0.0415


In [ ]:
'''
Procedimento para estimar o Modelo de Probabilidade Linear por MQGF (MQGF pois usamos h_chapeu)
Segue o item do livro: Estimando o modelo de probabilidade linear por mínimos quadrados ponderados
'''

In [8]:
# Passo 1: Estime o modelo por MQO e obtenha os valores estimados y_chapeu
modelo_passo_1 = smf.ols('PC ~ hsGPA + ACT + parcoll', data=df)
reg_passo_1 = modelo_passo_1.fit()
y_chapeu = reg_passo_1.fittedvalues

In [9]:
'''
Passo 2: Se todos os valores estimados (valores em y_chapeu) estiverem no intervalo aberto ]0, 1[, prossiga para o Passo 3.
         Se houver algum valor de y_chapeu fora do intervalo aberto ]0, 1[, abandone MQGF e estime o modelo por MQO
         utilizando as estatísticas robustas em relação à heterocedasticidade.
'''

teste = y_chapeu.between(0, 1, inclusive='neither')
if teste.sum() == teste.size:
    print('Todos os valores estimados estão no intervalo aberto ]0, 1[.\nProssiga para o passo 3')
else:
    print('Há valores em y_chapeu fora do intervalo aberto ]0, 1[.\
    \nEstime o modelo por MQO utilizando as estatísticas robustas em relação à heterocedasticidade.')

Todos os valores estimados estão no intervalo aberto ]0, 1[.
Prossiga para o passo 3


In [10]:
# Passo 3: Construa as variâncias estimadas na equação (8.47)
h_chapeu = y_chapeu * (1 - y_chapeu)

In [11]:
'''
Passo 4: Estime a equação do passo 1 por MQP usando peso = 1 / h_chapeu
'''      
peso = 1 / h_chapeu
modelo_mqgf = smf.wls('PC ~ hsGPA + ACT + parcoll', weights=peso, data=df)
reg_mqgf = modelo_mqgf.fit()
reg_mqgf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                     PC   R-squared:                       0.046
Model:                            WLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     2.224
Date:                Sun, 26 Jun 2022   Prob (F-statistic):             0.0882
Time:                        12:48:07   Log-Likelihood:                -95.812
No. Observations:                 141   AIC:                             199.6
Df Residuals:                     137   BIC:                             211.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0262      0.477      0.055      0.956      -0.916       0.969
hsGPA          0.0327      0.130      0.252      0.802      -0.224       0.290
ACT            0.0043      0.015      0.276      0.783      -0.026       0.035
parcoll        0.2152      0.086      2.494      0.014       0.045       0.386
==============================================================================
Omnibus:                    14585.053   Durbin-Watson:                   1.731
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.160
Skew:                           0.468   Prob(JB):                     6.91e-05
Kurtosis:                       1.455   Cond. No.                         293.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
'''
Observação sobre terminologia

Quando rodamos a regressão por MQO e por MQGF nomeamos variáveis como "modelo_ols" e "modelo_mqgf".
Nomeamos as variáveis desta forma pois o termo "modelo" é o mais usual, já os termos "ols" e "mqgf" apenas associam as 
variáveis aos métodos de estimativa utilizados.
Não queremos dizer que "estamos estimando um modelo MQO" ou um "modelo MQGF". Isto seria incorreto já que MQO e MQGF
são métodos de estimativa, não modelos.

Para detalhes, ver cap. 3 (Análise de regressão múltipla: Estimação), item 3.6 (Alguns comentários sobre a linguagem da
análise de regressão múltipla).
'''